In [4]:
import re, pickle, collections, numpy as np, keras, math, operator, pdb

#from gensim.models import word2vec, KeyedVectors
import matplotlib.pyplot as plt
import torch, torch.nn as nn
from torch.autograd import Variable
from torch import optim
from torchvision.models import inception_v3
import torch.nn.functional as F
from keras.preprocessing.sequence import pad_sequences
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.autograd import Variable
import re
from os import path
from glob import glob
from torch.utils.data import DataLoader
from PIL import Image
FLOAT_DTYPE = torch.cuda.FloatTensor
LONG_DTYPE = torch.cuda.LongTensor
from utils import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
dpath = 'flikr8/'
imgpath =  'flikr8/Flicker8k_Dataset/' 
tr_img_path = imgpath +'train/'
dev_img_path = imgpath +'dev/'
embed_path = 'glove.6B.300d.txt'

In [8]:
dev_caps = pickle.load(open(dpath+'dev_first_caps.pkl', 'rb'))
tr_caps = pickle.load(open(dpath+'train_first_caps.pkl', 'rb'))

dev_caps[0], tr_caps[0]
PAD = 0; SOS = 1; EOS = 2; UNK=3
re_apos = re.compile(r"(\w)'s\b")         # make 's a separate word
re_mw_punc = re.compile(r"(\w[’'])(\w)")  # other ' in a word creates 2 words
re_punc = re.compile("([\"().,;:/_?!—])") # add spaces around punctuation
re_mult_space = re.compile(r"  *")        # replace multiple spaces with just one


In [9]:
def simple_toks(sent):
    sent = re_apos.sub(r"\1 's", sent)
    sent = re_mw_punc.sub(r"\1 \2", sent)
    sent = re_punc.sub(r" \1 ", sent).replace('-', ' ')
    sent = re_mult_space.sub(' ', sent)
    return sent.lower().split()


def toks2ids(sents):
    #create new counter from all tokens in all lines
    sents_vals = list(sents.values())
    voc_cnt = collections.Counter(t for sent in sents_vals for t in sent) 
    
    #sort vocab in reverse order
    vocab = sorted(voc_cnt, key=voc_cnt.get, reverse=True)
    
    vocab.insert(PAD, "<PAD>")
    vocab.insert(SOS, "<SOS>")
    vocab.insert(EOS, "<EOS>")
    vocab.insert(UNK, "<UNK>")
    # {word: index of word}
    w2id = {w:i for i,w in enumerate(vocab)}
    #make each sentence into a list of ids
    ids = [[w2id[t] for t in sent] for sent in sents_vals]
    name_ids_dict = {name: id_vals for name, id_vals in zip(sents.keys(), ids)}
    return ids, vocab, w2id, voc_cnt, name_ids_dict


In [10]:
tokens = {name: simple_toks(words) for name, words in tr_caps}
dev_tokens = {name: simple_toks(words) for name, words in dev_caps}

keys = list(tokens.keys()); tokens[keys[0]]

['a',
 'child',
 'in',
 'a',
 'pink',
 'dress',
 'is',
 'climbing',
 'up',
 'a',
 'set',
 'of',
 'stairs',
 'in',
 'an',
 'entry',
 'way',
 '.']

In [11]:
ids, vocab, w2id, counts, name_ids_dict = toks2ids(tokens)
id2w = {v: k for k, v in w2id.items()}


In [ ]:
pickle.dump(ids, open(dpath+'tr_ids.pkl', 'wb'))
pickle.dump(vocab, open(dpath+'vocab.pkl', 'wb'))
pickle.dump(w2id, open(dpath+'w2id.pkl', 'wb'))
pickle.dump(name_ids_dict, open(dpath+'tr_name_ids_dict.pkl', 'wb'))
pickle.dump(id2w, open(dpath+'id2w.pkl', 'wb'))


In [18]:
def createDevNameIdsDict(sents, vocab):
    sents_vals = list(sents.values())    
    # {word: index of word}
    w2id = {w:i for i,w in enumerate(vocab)}
    #make each sentence into a list of ids
    ids = [[w2id.get(t, UNK) for t in sent] for sent in sents_vals]
    name_ids_dict = {name: id_vals for name, id_vals in zip(sents.keys(), ids)}
    return ids, name_ids_dict


In [19]:
dev_ids, dev_name_ids_dict = createValNameIdsDict(dev_tokens, vocab)

In [22]:
pickle.dump(dev_ids, open(dpath+'dev_ids.pkl', 'wb'))
pickle.dump(dev_name_ids_dict, open(dpath+'dev_name_ids_dict.pkl', 'wb'))

### Word vectors

In [3]:
embed_path = 'glove.6B.300d.txt'

def load_glove_me(loc):
    w2v = {}
    for line in open(loc):
        l = line.split()
        w2v[l[0]] = np.array(l[1:], dtype=np.float32)
    return w2v

w2v = load_glove_me(embed_path)
n_en_vec = len(w2v.keys()) #number of words encoded
dim_en_vec = len(w2v['the']) #dimension of embeddings -- we just use 'the' as a way to get it.

w2v['king'][:10]



def create_emb_mat(w2v, targ_vocab, dim_vec):
    vocab_size = len(targ_vocab)
    emb = np.zeros((vocab_size, dim_vec)) #initialize empty container
    found=0
    for i, word in enumerate(targ_vocab): 
        try: emb[i] = w2v[word]; found+=1 #if we find it, use the word's vecotrs
        except KeyError: emb[i] = np.random.normal(scale=0.6, size=(dim_vec,)) #else randomeness

    return emb, found

embs, found = create_emb_mat(w2v, vocab, dim_en_vec); embs.shape, found

In [13]:
pickle.dump(embs, open(dpath+'emb_mat_300.pkl', 'wb'))